# 🤖 Building a Simple Customer Support Agent

This notebook demonstrates how to build an autonomous agent using Claude and Tool Use (Function Calling). The agent can interact with a database to help customers with orders, refunds, and tracking.

## 🛠️ Core Capabilities

* **Business Logic Integration:** Directly connects to database helpers for customer data.
* **Intelligent Tool Selection:** The agent analyzes user intent to choose the right function.
* **Structured Data Processing:** Handles JSON inputs/outputs for reliable data flow.
* **Natural Language Generation:** Converts technical database results into friendly, human-style responses.

In [ ]:
%pip install anthropic python-dotenv

In [ ]:
# Setup with LiteLLM and .env
import os
from anthropic import Anthropic
from dotenv import load_dotenv
import json
import sqlite3
from datetime import datetime
from typing import Dict, Any

# Load environment variables from .env file
load_dotenv()

# Initialize Anthropic client (points to LiteLLM)
client = Anthropic(
    base_url=os.environ.get("ANTHROPIC_BASE_URL"),
    api_key=os.environ.get("ANTHROPIC_AUTH_TOKEN")
)

MODEL = os.environ.get("ANTHROPIC_MODEL", "claude-sonnet-4-20250514")

print("✅ Using Claude via LiteLLM")
print(f"🔗 Endpoint: {os.environ.get('ANTHROPIC_BASE_URL')}")
print(f"🤖 Model: {MODEL}")
print()

In [ ]:
# Initialize Database
import importlib
import db_helper
importlib.reload(db_helper)
from db_helper import reset_database

reset_database()

In [ ]:
# Agent Definition
from db_helper import (
    process_refund, 
    get_customer_orders, 
    check_order_status, 
    lookup_customer  # Added this import
)
from db_tools import tools

def customer_support_agent(user_message: str):
    """
    The main Agent Loop. It follows this flow:
    1. Receive user input.
    2. Call Claude with available tools.
    3. If Claude wants to use a tool, execute the Python function.
    4. Pass the result back to Claude.
    5. Repeat until Claude provides a final text answer.
    """
    
    messages = [{"role": "user", "content": user_message}]
    
    system_prompt = """You are a helpful customer support agent. You can:
                        - Look up customer information by customer ID
                        - Check order status for specific orders
                        - View all orders for a customer
                        - Process refunds for delivered orders
                        
                        Be polite, professional, and helpful. Always verify information before taking actions.
                        If you need to look up a customer's orders, ask for their customer ID first."""
    
    print(f"👤 Customer: {user_message}\n")
    
    # Initial call to the model
    response = client.messages.create(
        model=MODEL,
        max_tokens=1024,
        system=system_prompt,
        messages=messages,
        tools=tools
    )
    # print(response.content)
    
    # The "Agent Loop": Continue as long as the model wants to use tools
    while response.stop_reason == "tool_use":
        # Record the assistant's request to use a tool
        messages.append({
            "role": "assistant",
            "content": response.content
        })
        
        tool_results = []
        
        # Process each tool use request in the response
        for content_block in response.content:
            if content_block.type == "tool_use":
                tool_name = content_block.name
                tool_input = content_block.input
                
                print(f"🔍 Agent decided to use: {tool_name}")
                print(f"   Args: {tool_input}")
                
                # Routing logic: Map tool names to actual Python functions
                if tool_name == "lookup_customer":
                    result = lookup_customer(**tool_input)
                elif tool_name == "check_order_status":
                    result = check_order_status(**tool_input)
                elif tool_name == "get_customer_orders":
                    result = get_customer_orders(**tool_input)
                elif tool_name == "process_refund":
                    result = process_refund(**tool_input)
                else:
                    result = {"error": "Unknown function"}
                
                print(f"📊 Tool Result: {json.dumps(result, indent=2)}\n")
                
                # Format the result to send back to the model
                tool_results.append({
                    "type": "tool_result",
                    "tool_use_id": content_block.id,
                    "content": json.dumps(result)
                })
        
        # Add the tool execution results to the conversation history
        messages.append({
            "role": "user",
            "content": tool_results
        })
        
        # Call the model again with the new context
        response = client.messages.create(
            model=MODEL,
            max_tokens=1024,
            system=system_prompt,
            messages=messages,
            tools=tools
        )
        # print(response.content)
    
    # Final step: Extract and display the text response
    final_message = "".join([block.text for block in response.content if hasattr(block, "text")])
    
    print(f"🤖 Agent: {final_message}")
    return final_message

## 🧪 Testing Scenarios

### 1. Basic Information Retrieval

Testing if the agent can identify a single order status.

In [ ]:
# Test - Simple order lookup
customer_support_agent("Hi, can you check the status of order ORD12345?");

### 2. Multi-Step Reasoning & Action
Testing if the agent can look up a customer, identify their orders, filter for "Delivered" ones, and process a refund in a single conversation.

In [ ]:
# Test - Customer with all orders
customer_support_agent("My customer ID is CUST001. Can you show me all my orders?");

### 3. Conditional Logic: Refund Request
This test evaluates if the agent correctly identifies that a refund can only be processed if an order is in the "Delivered" state.

In [ ]:
# Test - Refund request
customer_support_agent("I want to return order ORD12347. The product didn't match the description.");


### 4. Complex Multi-Step Reasoning
This is the most advanced test for the agent. It requires the agent to:

Retrieve a list of all orders for a specific customer.

Parse that list to find items with a "Delivered" status.

Loop through those items to trigger individual refund processes.

In [ ]:
# Test - Complex multi-step
customer_support_agent("Hi, I'm customer CUST003. Show me my orders and process a refund for any delivered items.");